In [1]:

import numpy as np

class Random_walking_model():
    @staticmethod
    def TrueTrajectory(initial, diffs):
        array = [initial]
        for di in diffs:
            array.append(array[-1] + di)
        return np.array(array)

    def generate_signal(self, initial, sigma, n):
        diffs = np.random.normal(0, sigma, n-1)
        self.x = Random_walking_model.TrueTrajectory(initial, diffs)

    def __init__(self, initial, sigma, n):
        self.sigma = sigma
        self.generate_signal(initial, sigma, n)

def rvm_alpha(sig):
    '''linear regression koefficient(alpha)
    for random valking model process'''
    psi = sig[0]**2 / sig[1]**2

    alpha = (-psi + np.sqrt(psi**2 + 4 * psi)) / 2
    print('Optimal smoothing coefficient=', alpha)
    return alpha



In [2]:

# Assignment 3
# Determining and removing drawbacks of exponential and running mean.
# Team 2:
#     Ekaterina Karmanova
#     Timur Chikichev
#     Iaroslav Okunevich
#     Nikita Mikhailovskiy
#
# Skoltech, 04.10.2019

# import os
# try:
# 	os.chdir(os.path.join(os.getcwd(), 'lab3'))
# 	print(os.getcwd())
# except:
# 	pass
import numpy as np

class Smooth():
    def __init__(self, z):
        self.z = z

class RunningMean(Smooth):
    @staticmethod
    def smooth(z, m):
        x = np.zeros(len(z))
        sumFirst = 0
        sumLast = 0
        step = int((m-1)/2)
        for i in range(0, step):
            sumFirst += z[i]
            sumLast += z[len(z)-i-1]
        for i in range(step, len(z)-step):
            for j in range(i-step, i+step+1):
                x[i] += z[j]
            x[i] /= m
        for i in range(0, step):
            x[i] = sumFirst/step
            x[len(x)-i-1] = sumLast/step
        return x

    @staticmethod
    def window_width(alpha):
        return round((2-alpha)/alpha)

    # def calc_window_width(self, alpha):
    #     self.m = RunningMean.window_width(alpha)

    def __init__(self, z, alpha):
        super().__init__(z)
        # Smooth.__init__(z)
        self.m = RunningMean.window_width(alpha)
        # return m

    def run(self, m):
        ret = RunningMean.smooth(self.z, m)
        return ret

class FB_exp_smoothing(Smooth):
    @staticmethod
    def forward(alpfa, z):
        array = np.zeros(len(z))
        array[0] = 10
        for i in range(1, len(z)):
            array[i] = array[i - 1] + alpfa * (z[i] - array[i - 1])
        return array

    @staticmethod
    def backward(alpha, fz):
        array = np.zeros_like(fz)
        array[len(fz)-1] = fz[len(fz)-1]
        for i in range(len(fz)-2, -1, -1):
            array[i] = array[i+1]+alpha*(fz[i]-array[i+1])
        return array

    def run_f(self, alpha):
        res = FB_exp_smoothing.forward(alpha, self.z)
        return res

    def run_b(self, f, alpha):
        res = FB_exp_smoothing.backward(alpha, f)
        return res

    def run_fb(self, alpha):
        forward = FB_exp_smoothing.forward(alpha, self.z)
        res = FB_exp_smoothing.backward(alpha, forward)
        return res


In [4]:

# Assignment 3
# Determining and removing drawbacks of exponential and running mean.
# Team 2:
#     Ekaterina Karmanova
#     Timur Chikichev
#     Iaroslav Okunevich
#     Nikita Mikhailovskiy
#
# Skoltech, 04.10.2019

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#Assigment 3
#Part 1
# Array with рќњЋрќ‘¤2 and рќњЋрќњ‚2

class Measurements():
    def __init__(self, x, sigma):
        n = len(x)
        noise = np.random.normal(0, sigma, n)
        self.z = x + noise

    def DevVarInd(self, x):
        #find deviation and variability indicators.
        Id = 0
        Iv = 0
        z = self.z
        for i in range(len(z)):
            Id += (z[i]-x[i])**2
        for i in range(0, len(z)-2):
            Iv += (x[i+2] - 2*x[i+1] + x[i])**2
        return Id, Iv


def draw_plots(plots=[], show = False):
    for pi, plabel in plots:
        plt.plot(pi, label=plabel)

    plt.legend()
    if show:
        plt.show()

def new_plot(title, xl, yl, plots = [], show = False):
    figure(num=None, figsize=(10, 5), dpi=100,
           facecolor='w', edgecolor='k')
    plt.title(title)
    plt.ylabel(yl)
    plt.xlabel(xl)

    draw_plots(plots, show)

def part1():
    sigmasTrue = [28, 97]
    alpha = rvm_alpha(sigmasTrue)

    model = Random_walking_model(10, sigmasTrue[0], 300)
    x = model.x

    Known_vals = Measurements(x, sigmasTrue[1])
    z = Known_vals.z

    RM = sm.RunningMean(z, alpha)
    r300 = RM.run(RM.m)

    Expsmoothing = sm.FB_exp_smoothing(z)
    exp300 = Expsmoothing.run_f(alpha)
    bexp300 = Expsmoothing.run_fb(alpha)

    new_plot('Plot result with backward exponential mean',
             'Measurements', 'Points',
             [[r300, 'Smoothed by running mean'],
             [z, 'Measurements'],
             [x, 'True trajectory']], show=False)
    plt.savefig(fname='FB exp smoothing.png')


    def p1_3():
        #1.3
        indTru = Known_vals.DevVarInd(x)
        print('Indicator tru trajectory:\n', indTru)

        indEx = Known_vals.DevVarInd(exp300)
        print('Indicator forward expotnentially smoothing:\n', indEx)

        indBex = Known_vals.DevVarInd(bexp300)
        print('Indicator backward expotnentially smoothing:\n', indBex)

        indR = Known_vals.DevVarInd(r300)
        print('Indicator running smoothing:\n', indR)

        indZ = Known_vals.DevVarInd(z)
        print('Indicator measurements:\n', indZ)

    p1_3()

#Part II

def part2():
    a = np.random.normal(0, np.sqrt(10), 300)
    sig = np.sqrt(500)

    def Trajectory(size, acc, t):
        vel = Random_walking_model.TrueTrajectory(0, acc * t)
        trajectory = Random_walking_model.TrueTrajectory(0, acc * t * t / 2)

        new_plot('Plot result', 'Measurement', 'Points',
        [
            [acc, 'Acceleration'],
            [vel, 'Velosity'],
            [trajectory, 'Trajectory']
        ], show = False)
        plt.savefig(fname='Trajectory.png')
        return trajectory

    traject = Trajectory(300, a, 0.1)
    Known_vals = Measurements(traject, sig)
    z = Known_vals.z

    RM = sm.RunningMean(z, 1)

    new_plot('Plot result',
            'Measurements', 'Points', [
                [traject, 'Trajectory'],
                [z, 'Measure']
            ], show = False)
    plt.savefig(fname = 'running mean 1.png')

    def eval_params(z, chvals, b, run):
        krits = []
        for ai in chvals:
            r = run(ai)
            ki = Known_vals.DevVarInd(r)
            res = np.sum([ki[i]*b[i] for i in range(2)])
            print('a: {} ki: {} res: {}'.format(
            	ai, [ki[i]*b[i] for i in range(2)], res))
            krits.append(res)
        idmin = np.argmin(krits, 0)
        ret = chvals[idmin - 1: idmin + 2]
        return ret, krits[idmin]

    def plot_f(z, chvals, run, aname):
        new_plot('Plot result', 'Number', 'Count')
        plt.plot(z, label='Measure', c='lightgrey')
        for ai in chvals:
            r = run(ai)
            ki = Known_vals.DevVarInd(r)
            res = np.sum([ki[i]*b[i] for i in range(2)])
            plt.plot(r, label='{}, val: {}, res: {}'.format(aname, ai, res))
        plt.legend()
        # plt.show()

    n = 10
    b = [0.02, 0.08]
    chvals = np.round(np.linspace(0, z.shape[0] * 0.4, n))[1:]
    bestm, resm = eval_params(z, chvals, b, RM.run)
    plot_f(z, bestm, RM.run, 'Running mean')
    plt.savefig(fname='Running mean best.png')


    Expsmoothing = sm.FB_exp_smoothing(z)

    probe_alpha = np.linspace(0, 1, n + 2)[1:-1]
    best_alpha, resalpha = eval_params(z, probe_alpha, b, Expsmoothing.run_fb)

    print("bestm: {}, best_alpha: {}".format(bestm, best_alpha))
    plot_f(z, best_alpha, Expsmoothing.run_fb, 'exp fb smoothing')

    plt.savefig(fname='exp fb smoothing best.png')

    def proper_choise(m_vals, alpha_vals):
        m = m_vals
        a = alpha_vals

        res = 'exponential' if a < m else 'running mean'
        print("mean:{}, exp:{}".format(m_vals, alpha_vals))
        print(res)

    proper_choise(resm, resalpha)

def part3():
    #Second trajectory, 4
    def Trajectory(size, w, t):
            a = np.zeros((size))
            a[0] = 1
            trajectory = np.zeros((size))
            trajectory[0] = 5
            omega = 2*np.pi/t

            for i in range(1, size):
                    a[i] = a[i-1]+w[i]
            for i in range(0, size):
                    trajectory[i] = a[i]*np.sin(omega*i+3)
            return trajectory


    w = np.random.normal(0, 0.08, 200)
    traject = Trajectory(200, w, 32)

    #4
    ny = np.random.normal(0, np.sqrt(0.05), 200)
    z = Measurements(traject, ny)

    #5
    r = RunningMean(z, 13)

    figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
    plt.title('Plot result')
    plt.ylabel('Measurement')
    plt.xlabel('Points')
    plt.plot(traject, label='Trajectory')
    plt.plot(z, label='Measurements')
    plt.plot(r, label='Running mean')

    r = RunningMean(z, 27)
    figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
    plt.title('Plot result')
    plt.ylabel('Measurement')
    plt.xlabel('Points')
    plt.plot(traject, label='Trajectory')
    plt.plot(z, label='Measurements')
    plt.plot(r, label='Running mean')


def main():
    part1()
    part2()
    # part3()

if __name__ == "__main__":
    main()


Optimal smoothing coefficient= 0.24998861233121078


NameError: name 'sm' is not defined